<a href="https://colab.research.google.com/github/patelsaraha/Fetch-Data-Project/blob/main/ExploreTheData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

# File paths
user_file = "/content/USER_TAKEHOME.csv"
transaction_file = "/content/TRANSACTION_TAKEHOME.csv"
product_file = "/content/PRODUCTS_TAKEHOME.csv"

# Load data
df_users = pd.read_csv(user_file)
df_transactions = pd.read_csv(transaction_file)
df_products = pd.read_csv(product_file)

# Display basic info about each dataset
df_users.info(), df_transactions.info(), df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   ID            100000 non-null  object
 1   CREATED_DATE  100000 non-null  object
 2   BIRTH_DATE    96325 non-null   object
 3   STATE         95188 non-null   object
 4   LANGUAGE      69492 non-null   object
 5   GENDER        94108 non-null   object
dtypes: object(6)
memory usage: 4.6+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   RECEIPT_ID      50000 non-null  object 
 1   PURCHASE_DATE   50000 non-null  object 
 2   SCAN_DATE       50000 non-null  object 
 3   STORE_NAME      50000 non-null  object 
 4   USER_ID         50000 non-null  object 
 5   BARCODE         44238 non-null  float64
 6   FINAL_QUANTITY  50000 non-null  ob

(None, None, None)

In [4]:
# Check for duplicate records in each dataset
duplicate_users = df_users.duplicated().sum()
duplicate_transactions = df_transactions.duplicated().sum()
duplicate_products = df_products.duplicated().sum()

# Check unique values for key fields
unique_users = df_users["ID"].nunique()
unique_transactions = df_transactions["RECEIPT_ID"].nunique()
unique_products = df_products["BARCODE"].nunique()

# Check for inconsistencies in barcode data (e.g., non-null in transactions but missing in products)
barcode_mismatch = df_transactions["BARCODE"].dropna().nunique() - df_products["BARCODE"].dropna().nunique()

duplicate_users, duplicate_transactions, duplicate_products, unique_users, unique_transactions, unique_products, barcode_mismatch


(0, 171, 57, 100000, 24440, 410517, -399490)

In [5]:
# Convert date columns to datetime format
df_users["CREATED_DATE"] = pd.to_datetime(df_users["CREATED_DATE"], errors="coerce")
df_users["BIRTH_DATE"] = pd.to_datetime(df_users["BIRTH_DATE"], errors="coerce")

df_transactions["PURCHASE_DATE"] = pd.to_datetime(df_transactions["PURCHASE_DATE"], errors="coerce")
df_transactions["SCAN_DATE"] = pd.to_datetime(df_transactions["SCAN_DATE"], errors="coerce")

# Convert numerical fields to appropriate types
df_transactions["FINAL_QUANTITY"] = pd.to_numeric(df_transactions["FINAL_QUANTITY"], errors="coerce")
df_transactions["FINAL_SALE"] = pd.to_numeric(df_transactions["FINAL_SALE"], errors="coerce")

# Convert barcode to string for consistency
df_transactions["BARCODE"] = df_transactions["BARCODE"].astype(str)
df_products["BARCODE"] = df_products["BARCODE"].astype(str)

# Drop duplicates
df_transactions = df_transactions.drop_duplicates()
df_products = df_products.drop_duplicates()

# Check final dataset info after cleanup
df_users.info(), df_transactions.info(), df_products.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype              
---  ------        --------------   -----              
 0   ID            100000 non-null  object             
 1   CREATED_DATE  100000 non-null  datetime64[ns, UTC]
 2   BIRTH_DATE    96325 non-null   datetime64[ns, UTC]
 3   STATE         95188 non-null   object             
 4   LANGUAGE      69492 non-null   object             
 5   GENDER        94108 non-null   object             
dtypes: datetime64[ns, UTC](2), object(4)
memory usage: 4.6+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 49829 entries, 0 to 49999
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   RECEIPT_ID      49829 non-null  object             
 1   PURCHASE_DATE   49829 non-null  datetime64[ns]     
 2   SCAN_DATE       49829 non-null  datetime64[

(None, None, None)

In [6]:
from datetime import datetime

# Calculate user ages
current_year = datetime.now().year
df_users["AGE"] = current_year - df_users["BIRTH_DATE"].dt.year

# Filter users 21 and over
df_users_21_plus = df_users[df_users["AGE"] >= 21]

# Join transactions with users and products
df_merged = df_transactions.merge(df_users_21_plus, left_on="USER_ID", right_on="ID", how="inner")
df_merged = df_merged.merge(df_products, on="BARCODE", how="left")

# Count receipts scanned per brand
top_brands = df_merged.groupby("BRAND")["RECEIPT_ID"].nunique().sort_values(ascending=False).head(5)
top_brands


,RECEIPT_ID
BRAND,
NERDS CANDY,13
SPEED STICK,12
TOSTITOS,12
ENTENMANN'S SWEET BAKED GOODS,12
COCA-COLA,12


In [7]:
from datetime import datetime
import pandas as pd

# Load data
df_users = pd.read_csv("USER_TAKEHOME.csv")
df_transactions = pd.read_csv("TRANSACTION_TAKEHOME.csv")
df_products = pd.read_csv("PRODUCTS_TAKEHOME.csv")

# Convert date columns
df_users["BIRTH_DATE"] = pd.to_datetime(df_users["BIRTH_DATE"], errors="coerce")

# Calculate user ages
current_year = datetime.now().year
df_users["AGE"] = current_year - df_users["BIRTH_DATE"].dt.year

# Filter users 21 and over
df_users_21_plus = df_users[df_users["AGE"] >= 21]

# Convert barcode to string for consistency
df_transactions["BARCODE"] = df_transactions["BARCODE"].astype(str)
df_products["BARCODE"] = df_products["BARCODE"].astype(str)

# Join transactions with users and products
df_merged = df_transactions.merge(df_users_21_plus, left_on="USER_ID", right_on="ID", how="inner")
df_merged = df_merged.merge(df_products, on="BARCODE", how="left")

# Count receipts scanned per brand
top_brands = df_merged.groupby("BRAND")["RECEIPT_ID"].nunique().sort_values(ascending=False).head(5)
print(top_brands)


BRAND
NERDS CANDY        14
DOVE               14
COCA-COLA          13
SOUR PATCH KIDS    13
HERSHEY'S          13
Name: RECEIPT_ID, dtype: int64
